In [1]:
#n.p check input at native or standard, check mask as native or standard, check time series, check parameters too of course

In [3]:
import os
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import image
from nilearn.glm.first_level import compute_regressor
import logging
from brainiak.searchlight.searchlight import Searchlight, Ball

from statsmodels.tsa.stattools import grangercausalitytests
from scipy import stats
import sys
from mpi4py import MPI
import gc
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import your parameters
curr_dir = '/user_data/csimmon2/git_repos/ptoc'
import sys
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
localizer = 'Scramble'  # scramble or object. This is the localizer task.
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
raw_dir = params.raw_dir

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
sub_info = sub_info[sub_info['group'] == 'control']
subs = sub_info['sub'].tolist()
subs = ['sub-025']

run_num = 3
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) for rn2 in range(rn1 + 1, run_num + 1)]

# Searchlight parameters
searchlight_radius = 2  # in voxels, adjust as needed
max_blk_edge = 10
pool_size = 1

def load_and_prepare_data(sub, run_combo):
    """
    Load and prepare data for a single subject and run combination.
    
    :param sub: Subject ID
    :param run_combo: List of run numbers to combine
    :return: Tuple of (4D fMRI data, brain mask, psychological covariate)
    """
    logging.info(f"Loading data for subject {sub}, runs {run_combo}")
    
    # Load and combine run data
    run_data_list = []
    for run in run_combo:
        run_file = f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-0{run}/1stLevel.feat/filtered_func_data_reg.nii.gz' #registered but not standardaized
        run_img = image.load_img(run_file)
        run_data = image.clean_img(run_img, standardize=True)
        run_data_list.append(run_data)
    
    # Concatenate run data
    fmri_data = image.concat_imgs(run_data_list)
    
    # Load brain mask
    mask_file = f'{study_dir}/{sub}/ses-01/derivatives/rois/parcels/pIPS.nii.gz'
    brain_mask = nib.load(mask_file).get_fdata().astype(bool)
    
    # Generate psychological covariate
    psy = make_psy_cov(run_combo, sub)
    
    # Ensure fMRI data and psy covariate have the same number of time points
    if fmri_data.shape[-1] != len(psy):
        raise ValueError(f"Mismatch in volumes: fMRI data has {fmri_data.shape[-1]}, psy has {len(psy)}")
    
    # Convert fmri_data to 4D numpy array if it's not already
    if isinstance(fmri_data, nib.Nifti1Image):
        fmri_data = fmri_data.get_fdata()
    
    # Ensure fmri_data is 4D
    if fmri_data.ndim != 4:
        raise ValueError(f"fMRI data must be 4D, but got shape {fmri_data.shape}")
    
    return fmri_data, brain_mask, psy

def make_psy_cov(runs, ss):
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    vols_per_run, tr = 184, 2.0
    total_vols = vols_per_run * len(runs)
    times = np.arange(0, total_vols * tr, tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for i, rn in enumerate(runs):
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_{localizer}.txt'

        if not os.path.exists(obj_cov_file):
            logging.warning(f'Covariate file not found for run {rn}')
            continue

        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        
        if i > 0:
            obj_cov['onset'] += i * vols_per_run * tr
        
        full_cov = pd.concat([full_cov, obj_cov])

    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    cov = full_cov.to_numpy()
    valid_onsets = cov[:, 0] < times[-1]
    cov = cov[valid_onsets]

    if cov.shape[0] == 0:
        logging.warning('No valid covariate data after filtering. Returning zeros array.')
        return np.zeros((total_vols, 1))

    psy, _ = compute_regressor(cov.T, 'spm', times)
    psy[psy > 0] = 1
    psy[psy <= 0] = 0
    return psy

In [ ]:
# Global variable declaration
global psy

def gca_measure(data, mask, myrad, bcvar):
    global psy
    
    data_4d = data[0]
    data_2d = data_4d.reshape(data_4d.shape[0], -1)
    center_ts = data_2d[:, 0]
    other_ts = np.mean(data_2d[:, 1:], axis=1)
    
    gc_center_to_other = grangercausalitytests(np.column_stack((other_ts, center_ts, psy.flatten())), maxlag=1, verbose=False)
    gc_other_to_center = grangercausalitytests(np.column_stack((center_ts, other_ts, psy.flatten())), maxlag=1, verbose=False)
    
    f_diff = gc_center_to_other[1][0]['ssr_ftest'][0] - gc_other_to_center[1][0]['ssr_ftest'][0]
    
    return f_diff

def run_searchlight(fmri_data, brain_mask):
    assert brain_mask.ndim == 3 and brain_mask.dtype == bool, "Invalid brain_mask"

    sl = Searchlight(sl_rad=searchlight_radius, max_blk_edge=10)
    fmri_data_4d = fmri_data.transpose(3, 0, 1, 2)
    fmri_data_list = [fmri_data_4d]
    
    sl.distribute(fmri_data_list, brain_mask)
    sl_result = sl.run_searchlight(gca_measure, pool_size=1)
    
    return sl_result

if __name__ == "__main__":
    global psy
    
    for sub in subs:
        for run_combo in run_combos:
            try:
                fmri_data, brain_mask, psy = load_and_prepare_data(sub, run_combo)
                print(f"Processing subject {sub}, runs {run_combo}")
                print(f"Data shapes - fMRI: {fmri_data.shape}, Mask: {brain_mask.shape}, PSY: {psy.shape}")
                
                assert psy.shape[0] == fmri_data.shape[3], "PSY shape mismatch"
                
                sl_result = run_searchlight(fmri_data, brain_mask)
                
                result_file = f'{results_dir}/searchlight_gca_{sub}_runs{"_".join(map(str, run_combo))}.nii.gz'
                nib.save(nib.Nifti1Image(sl_result, affine=nib.load(f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-01/1stLevel.feat/filtered_func_data_reg.nii.gz').affine), result_file)
                
                print(f"Completed analysis for subject {sub}, runs {run_combo}")
                
            except AssertionError as e:
                print(f"Assertion error for subject {sub}, runs {run_combo}: {str(e)}")
            except Exception as e:
                print(f"Error processing subject {sub}, runs {run_combo}: {str(e)}")

In [ ]:
#HELP 10.7.24
import os
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import image, input_data
from nilearn.glm.first_level import compute_regressor
import logging
from brainiak.searchlight.searchlight import Searchlight, Ball
from statsmodels.tsa.stattools import grangercausalitytests
import sys
import gc
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import your parameters
curr_dir = '/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
localizer = 'Object'  # scramble or object. This is the localizer task.
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
raw_dir = params.raw_dir

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
sub_info = sub_info[sub_info['group'] == 'control']
#subs = sub_info['sub'].tolist()
subs = ['sub-025']

rois = ['pIPS', 'LO']
hemispheres = ['left', 'right']
run_num = 3
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) for rn2 in range(rn1 + 1, run_num + 1)]

# Searchlight parameters
searchlight_radius = 2 
max_blk_edge = 10
pool_size = 1

def extract_roi_sphere(img, coords):
    roi_masker = input_data.NiftiSpheresMasker([tuple(coords)], radius=6)
    seed_time_series = roi_masker.fit_transform(img)
    phys = np.mean(seed_time_series, axis=1).reshape(-1, 1)
    return phys

def make_psy_cov(runs, ss):
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    vols_per_run, tr = 184, 2.0
    total_vols = vols_per_run * len(runs)
    times = np.arange(0, total_vols * tr, tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for i, rn in enumerate(runs):
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_{localizer}.txt'

        if not os.path.exists(obj_cov_file):
            logging.warning(f'Covariate file not found for run {rn}')
            continue

        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        
        if i > 0:
            obj_cov['onset'] += i * vols_per_run * tr
        
        full_cov = pd.concat([full_cov, obj_cov])

    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    cov = full_cov.to_numpy()
    valid_onsets = cov[:, 0] < times[-1]
    cov = cov[valid_onsets]

    if cov.shape[0] == 0:
        logging.warning('No valid covariate data after filtering. Returning zeros array.')
        return np.zeros((total_vols, 1))

    psy, _ = compute_regressor(cov.T, 'spm', times)
    psy[psy > 0] = 1
    psy[psy <= 0] = 0
    return psy

def extract_cond_ts(ts, cov):
    block_ind = (cov==1)
    block_ind = np.insert(block_ind, 0, True)
    block_ind = np.delete(block_ind, len(block_ind)-1)
    block_ind = (cov == 1).reshape((len(cov))) | block_ind
    return ts[block_ind]

from nilearn import image

def load_and_prepare_data(sub, run_combo):
    logging.info(f"Loading data for subject {sub}, runs {run_combo}")
    
    # Load whole-brain mask (assumed to be in standard space)
    whole_brain_mask_file = f'/user_data/csimmon2/git_repos/ptoc/roiParcels/mruczek_parcels/binary/all_visual_areas.nii.gz'
    if not os.path.exists(whole_brain_mask_file):
        raise FileNotFoundError(f"Whole-brain mask file not found: {whole_brain_mask_file}")
    whole_brain_mask = nib.load(whole_brain_mask_file)
    
    filtered_list = []
    for run in run_combo:
        run_file = f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-0{run}/1stLevel.feat/filtered_func_data_reg.nii.gz'
        if not os.path.exists(run_file):
            raise FileNotFoundError(f"Run file not found: {run_file}")
        
        # Load the current run
        curr_run = nib.load(run_file)
        
        # Standardize the fMRI data to match the space of the whole-brain mask
        # You may need to adjust the target_affine and target_shape based on your specific standardization requirements
        standardized_run = image.resample_img(curr_run, 
                                              target_affine=whole_brain_mask.affine, 
                                              target_shape=whole_brain_mask.shape,
                                              interpolation='continuous')
        
        # Now clean and mask the standardized data
        cleaned_run = image.clean_img(standardized_run, standardize=True, mask_img=whole_brain_mask)
        
        # Get the data as a numpy array
        cleaned_data = image.get_data(cleaned_run)
        
        filtered_list.append(cleaned_data)
    
    # Concatenate the runs
    fmri_data_array = np.concatenate(filtered_list, axis=-1)
    
    # Create a 4D image for use with nilearn functions
    img4d = nib.Nifti1Image(fmri_data_array, whole_brain_mask.affine)
    
    return fmri_data_array, whole_brain_mask.get_fdata().astype(bool), img4d


def gca_measure(data, mask, myrad, seed_ts, psy):
    data_4d = data[0]
    data_2d = data_4d.reshape(data_4d.shape[0], -1)
    center_ts = data_2d[:, 0]
    
    center_ts_cond = extract_cond_ts(center_ts, psy)
    seed_ts_cond = extract_cond_ts(seed_ts, psy)
    
    if np.allclose(center_ts_cond, center_ts_cond[0]) or np.allclose(seed_ts_cond, seed_ts_cond[0]):
        return 0
    
    try:
        gc_center_to_seed = grangercausalitytests(np.column_stack((seed_ts_cond, center_ts_cond)), maxlag=1, verbose=False)
        gc_seed_to_center = grangercausalitytests(np.column_stack((center_ts_cond, seed_ts_cond)), maxlag=1, verbose=False)
        f_diff = gc_center_to_seed[1][0]['ssr_ftest'][0] - gc_seed_to_center[1][0]['ssr_ftest'][0]
    except Exception as e:
        logging.warning(f"Error in Granger causality test: {str(e)}. Returning 0 for this voxel.")
        return 0
    
    return f_diff

def run_searchlight(fmri_data, brain_mask, seed_ts, psy):
    sl = Searchlight(sl_rad=searchlight_radius, max_blk_edge=max_blk_edge, shape=Ball)
    sl.distribute([fmri_data], brain_mask)
    sl.broadcast(seed_ts)
    sl.broadcast(psy)
    sl_result = sl.run_searchlight(gca_measure, pool_size=pool_size)
    return sl_result

def conduct_gca_searchlight():
    logging.info(f'Running GCA Searchlight for {localizer}...')
    
    for ss in subs:
        sub_dir = f'{study_dir}/{ss}/ses-01/'
        roi_dir = f'{sub_dir}/derivatives/rois'
        os.makedirs(f'{sub_dir}/derivatives/gca_searchlight', exist_ok=True)

        roi_coords = pd.read_csv(f'{roi_dir}/spheres/sphere_coords_std.csv')
        logging.info(f"ROI coordinates loaded for subject {ss}")

        for rcn, rc in enumerate(run_combos):
            logging.info(f"Processing run combination {rc} for subject {ss}")
            
            fmri_data, brain_mask, img4d = load_and_prepare_data(ss, rc)
            psy = make_psy_cov(rc, ss)
            
            if fmri_data.shape[3] != psy.shape[0]:
                raise ValueError(f"Mismatch in volumes: fMRI data has {fmri_data.shape[3]}, psy has {psy.shape[0]}")
            
            for dorsal_roi in ['pIPS']:
                for dorsal_hemi in hemispheres:
                    dorsal_coords = roi_coords[(roi_coords['index'] == rcn) & 
                                               (roi_coords['roi'] == dorsal_roi) &
                                               (roi_coords['hemisphere'] == dorsal_hemi)]
                    
                    if dorsal_coords.empty:
                        logging.warning(f"No coordinates found for {dorsal_roi}, {dorsal_hemi}, run combo {rc}")
                        continue

                    dorsal_ts = extract_roi_sphere(img4d, dorsal_coords[['x', 'y', 'z']].values.tolist()[0])
                    
                    t1 = time.time()
                    logging.info(f"Begin Searchlight for {ss}, {dorsal_roi}_{dorsal_hemi}, run combo {rc}")
                    sl_result = run_searchlight(fmri_data, brain_mask, dorsal_ts, psy)
                    logging.info(f"End Searchlight - Duration: {(time.time()-t1)/60:.2f} minutes")
                    
                    # Save results
                    sl_result = sl_result.astype('double')
                    sl_result[np.isnan(sl_result)] = 0
                    affine = img4d.affine
                    sl_nii = nib.Nifti1Image(sl_result, affine)
                    output_file = f'{sub_dir}/derivatives/gca_searchlight/searchlight_gca_{ss}_{dorsal_hemi}_{dorsal_roi}_seed_rc{rcn}_{localizer.lower()}.nii.gz'
                    nib.save(sl_nii, output_file)
                    logging.info(f"Results saved to {output_file}")

if __name__ == "__main__":
    conduct_gca_searchlight()

2024-10-07 13:30:49,475 - INFO - Running GCA Searchlight for Object...
2024-10-07 13:30:49,482 - INFO - ROI coordinates loaded for subject sub-025
2024-10-07 13:30:49,483 - INFO - Processing run combination [1, 2] for subject sub-025
2024-10-07 13:30:49,485 - INFO - Loading data for subject sub-025, runs [1, 2]
